In [38]:
# coding=utf-8
# Título: Realizar limpieza de la dirección catastral de Bogotá con Python, Arcpy y VStudio Code
# Requerimientos: ArcGIS Pro, Python 3x
# Librerías: arcpy, time, os
# Autor: Equipo SIG / DATA | ACTED - REACH
# Fecha: 2025-03-21

### Importar librerías

In [39]:
import pandas as pd

### Cargar el archivo y crear dataframe

In [41]:
# Ruta archivo
archivo = r"D:\ACTED\IMPACT COL - General\5_GIS\2_Projects\2025_XXX_ECHO\ABA\BOG-BELLAFLOR\2_workspace\catastro\bellaflor_sur_catastro_dir_20250321.csv"

# Leer archivo
df = pd.read_csv(archivo,delimiter=';')

print("Dataframe creado!")

# Guardar archivo
#df.to_csv(r"D:\ACTED\IMPACT COL - General\5_GIS\2_Projects\2025_XXX_ECHO\ABA\BOG-BELLAFLOR\2_workspace\catastro\bellaflor_sur_catastro_dir_20250321.csv", index = False)

Dataframe creado!


### Revisar el dataframe creado

In [42]:
# Revisar
df.head(3)

,OBJECTID,LotCodigo,LotCodMzn,id,longitud,latitud,PreDirecc
0,1,2580022019,2580022,7,"-74,160964950999983","4,546422568000025",KR 27B 72D 20 SUR MJ
1,2,2580068018,2580068,1068,"-74,160459671999945","4,543975258000046",CL 73C BIS A SUR 26F 03 MJ
2,3,2580012009,2580012,1144,"-74,162226311999973","4,544696038000041",CL 73D BIS B SUR 27B 31


### Limpiar columnas de interés

In [45]:
# Limpiar columnas
#df = df.drop(['OBJECTID'],axis=1)
#print(f'Después:\n {df.columns}')
df.head(5)

,LotCodigo,LotCodMzn,id,longitud,latitud,PreDirecc
0,2580022019,2580022,7,"-74,160964950999983","4,546422568000025",KR 27B 72D 20 SUR MJ
1,2580068018,2580068,1068,"-74,160459671999945","4,543975258000046",CL 73C BIS A SUR 26F 03 MJ
2,2580012009,2580012,1144,"-74,162226311999973","4,544696038000041",CL 73D BIS B SUR 27B 31
3,2580012004,2580012,1147,"-74,162241627999947","4,544575396000027",CL 74 SUR 27B 24
4,2580012008,2580012,1148,"-74,162285076999979","4,544728080000027",CL 73D BIS B SUR 27B 37


### Crear campo para contar espacios en cada fila

In [46]:
# Crea un nuevo campo que cuente el número de espacios en cada fila del campo PreDirecc
df['num_espacios'] = df['PreDirecc'].str.count(' ') + 1
df.head(5)

,LotCodigo,LotCodMzn,id,longitud,latitud,PreDirecc,num_espacios
0,2580022019,2580022,7,"-74,160964950999983","4,546422568000025",KR 27B 72D 20 SUR MJ,6
1,2580068018,2580068,1068,"-74,160459671999945","4,543975258000046",CL 73C BIS A SUR 26F 03 MJ,8
2,2580012009,2580012,1144,"-74,162226311999973","4,544696038000041",CL 73D BIS B SUR 27B 31,7
3,2580012004,2580012,1147,"-74,162241627999947","4,544575396000027",CL 74 SUR 27B 24,5
4,2580012008,2580012,1148,"-74,162285076999979","4,544728080000027",CL 73D BIS B SUR 27B 37,7


### Crear campos a partir del número de espacios

In [47]:
# Crea una lista de campos dir_ nuevos con el mismo número de elementos que el valor de n
max_n = df['num_espacios'].max()
dirs_cols = [f'dir_{i+1}' for i in range(max_n)]

print(f"Número de campos: {max_n}")
print(f"Campos creados: {dirs_cols}")

Número de campos: 11
Campos creados: ['dir_1', 'dir_2', 'dir_3', 'dir_4', 'dir_5', 'dir_6', 'dir_7', 'dir_8', 'dir_9', 'dir_10', 'dir_11']


### Aplicar la función para separar y crear campos por espacio " "

In [48]:
# Utiliza la función apply para aplicar la función str.split a cada fila del campo PreDirecc
df[dirs_cols] = df.apply(lambda row: pd.Series(row['PreDirecc'].split(' ', row['num_espacios'])), axis=1)
df.head(5)

,LotCodigo,LotCodMzn,id,longitud,latitud,PreDirecc,num_espacios,dir_1,dir_2,dir_3,dir_4,dir_5,dir_6,dir_7,dir_8,dir_9,dir_10,dir_11
0,2580022019,2580022,7,"-74,160964950999983","4,546422568000025",KR 27B 72D 20 SUR MJ,6,KR,27B,72D,20,SUR,MJ,NaN,NaN,NaN,NaN,NaN
1,2580068018,2580068,1068,"-74,160459671999945","4,543975258000046",CL 73C BIS A SUR 26F 03 MJ,8,CL,73C,BIS,A,SUR,26F,03,MJ,NaN,NaN,NaN
2,2580012009,2580012,1144,"-74,162226311999973","4,544696038000041",CL 73D BIS B SUR 27B 31,7,CL,73D,BIS,B,SUR,27B,31,NaN,NaN,NaN,NaN
3,2580012004,2580012,1147,"-74,162241627999947","4,544575396000027",CL 74 SUR 27B 24,5,CL,74,SUR,27B,24,NaN,NaN,NaN,NaN,NaN,NaN
4,2580012008,2580012,1148,"-74,162285076999979","4,544728080000027",CL 73D BIS B SUR 27B 37,7,CL,73D,BIS,B,SUR,27B,37,NaN,NaN,NaN,NaN


### Función para describir variables y frecuencia de valores

In [49]:
columna_excluir = ['LotCodigo', 'LotCodMzn', 'id', 'longitud', 'latitud','PreDirecc','num_espacios']

# f. Describir contenido de celdas
def mostrar_contenido(data):
    """
    Función que enseña el contenido de cada celda y columna en el df
    """
    for column in data.columns:
        if column not in columna_excluir:
            unique_values = data[column].unique()
            unique_values = sorted(unique_values, key=lambda x: str(x) or '')  # Ordenar los valores alfabéticamente incluyendo nulos o valores en blanco
            print(f"Valores únicos de la columna {column.upper()} | Total = {len(unique_values)} \n{unique_values}\n")
            print(f"Valores únicos y su frecuencia:\n{data[column].value_counts()}\n")
        else:
            print(f"Columna excluida: {column}")

mostrar_contenido(df)



Columna excluida: LotCodigo
Columna excluida: LotCodMzn
Columna excluida: id
Columna excluida: longitud
Columna excluida: latitud
Columna excluida: PreDirecc
Columna excluida: num_espacios
Valores únicos de la columna DIR_1 | Total = 5 
['CL', 'DG', 'KR', 'PTLT', 'TV']

Valores únicos y su frecuencia:
dir_1
KR      789
CL      606
DG       97
TV       62
PTLT      1
Name: count, dtype: int64

Valores únicos de la columna DIR_2 | Total = 53 
['18R', '19D', '20A', '23A', '24', '26', '26A', '26BIS', '26BISB', '26C', '26D', '26F', '26G', '27', '27A', '27B', '27C', '27F', '27G', '27H', '46', '69L', '7', '70', '70A', '70D', '70DBIS', '70F', '70G', '70I', '70X', '71A', '72', '72C', '72D', '72F', '72G', '72H', '72I', '72J', '73', '73A', '73B', '73C', '73D', '74', '74A', '74B', '74C', '75', '76B', '77', '78']

Valores únicos y su frecuencia:
dir_2
26        169
27        160
26A       134
73C       126
73D       111
73         78
73A        68
27B        68
27A        66
73B        64
26C      